# Geometrical accuracy

 
This notebook explains a direct method to generate the lower body cartilage surface mesh based on the bone geometries
    

# imports

In [1]:
import numpy as np
import meshplot as mp
from pathlib import Path
import sys
sys.path.append('../')
import os
import src
import igl
import time

In [2]:
"""
VALUES:

* i_dim, o_dim = input and output dimension ("mm" = millimeters, "m" = meters)

* o_format = format you want the files to be save at ( ".obj" , ".stl")

* + scroll down to calibrate the cartilage generation parameters
"""
# is the model coarse?
coarse = False
model_id = 'm8'
if coarse:
    subject_id = model_id + '_coarse'
else: 
    subject_id = model_id + '_fine'
    
# user-defined names
sacrum_name  = 'sacrum'
lpelvis_name = 'lpelvis'
rpelvis_name = 'rpelvis'
lfemur_name  = 'lfemur' 
rfemur_name  = 'rfemur'
#
lsi_name = 'lsacroiliac'
rsi_name = 'rsacroiliac'
lpelvic_name = 'lhip_ac_w_gap'
rpelvic_name = 'rhip_ac_w_gap'
lfemoral_name = 'lhip_fc_w_gap'
rfemoral_name = 'rhip_fc_w_gap'
pubic_name = 'pubic'

# dimensions 
i_dim  = "mm"     

# suffix
i_format = ".stl"
i2_format = ".obj"
o_format = ".obj"


In [3]:
"""
DIRECTORIES
"""
main_dir = Path('..')

i_dir_surf_bone  = main_dir / 'models'/ 'SurfaceMesh'/ 'cargen_output'/ model_id
i_dir_surf_cart  = main_dir / 'models'/ 'SurfaceMesh'/ 'cargen_output'/ model_id

i_dir_vol= main_dir / 'models'/ 'VolumeMesh'/ 'volgen_output'/ subject_id


In [4]:
"""
PATHS:
"""

# input surf 
# bone
# input_s_sacrum  = model_id + '_'+ sacrum_name  + '_' + i_dim
# input_s_lpelvis = model_id + '_'+ lpelvis_name + '_' + i_dim
# input_s_rpelvis = model_id + '_'+ rpelvis_name + '_' + i_dim
# input_s_lfemur  = model_id + '_'+ lfemur_name  + '_' + i_dim
# input_s_rfemur  = model_id + '_'+ rfemur_name  + '_' + i_dim

input_s_sacrum  = model_id + '_cg_bone_'+ sacrum_name  
input_s_lpelvis = model_id + '_cg_bone_'+ lpelvis_name 
input_s_rpelvis = model_id + '_cg_bone_'+ rpelvis_name 
input_s_lfemur  = model_id + '_cg_bone_'+ lfemur_name  
input_s_rfemur  = model_id + '_cg_bone_'+ rfemur_name  
input_s_sacrum_path  = str((i_dir_surf_bone /input_s_sacrum).with_suffix(i2_format))
input_s_lpelvis_path = str((i_dir_surf_bone /input_s_lpelvis).with_suffix(i2_format))
input_s_rpelvis_path = str((i_dir_surf_bone /input_s_rpelvis).with_suffix(i2_format))
input_s_lfemur_path  = str((i_dir_surf_bone /input_s_lfemur).with_suffix(i2_format))
input_s_rfemur_path  = str((i_dir_surf_bone /input_s_rfemur).with_suffix(i2_format))

# carts
input_s_lsi      = model_id + '_cg_jnt_' + lsi_name  
input_s_rsi      = model_id + '_cg_jnt_' + rsi_name 
input_s_lpelvic  = model_id + '_cg_jnt_' + lpelvic_name 
input_s_rpelvic  = model_id + '_cg_jnt_' + rpelvic_name 
input_s_lfemoral = model_id + '_cg_jnt_' + lfemoral_name 
input_s_rfemoral = model_id + '_cg_jnt_' + rfemoral_name 
input_s_pubic    = model_id + '_cg_jnt_' + pubic_name  

input_s_lsi_path      = str((i_dir_surf_cart /input_s_lsi).with_suffix(i2_format))
input_s_rsi_path      = str((i_dir_surf_cart /input_s_rsi).with_suffix(i2_format))
input_s_lpelvic_path  = str((i_dir_surf_cart /input_s_lpelvic).with_suffix(i2_format))
input_s_rpelvic_path  = str((i_dir_surf_cart /input_s_rpelvic).with_suffix(i2_format))
input_s_lfemoral_path = str((i_dir_surf_cart /input_s_lfemoral).with_suffix(i2_format))
input_s_rfemoral_path = str((i_dir_surf_cart /input_s_rfemoral).with_suffix(i2_format))
input_s_pubic_path    = str((i_dir_surf_cart /input_s_pubic).with_suffix(i2_format))

# input vol
input_v_sacrum  = model_id + '_girdle_w_gap_sacrum'
input_v_lpelvis = model_id + '_girdle_w_gap_lpelvis'
input_v_rpelvis = model_id + '_girdle_w_gap_rpelvis'
input_v_lfemur  = model_id + '_left_leg_w_gap_femur'
input_v_rfemur  = model_id + '_right_leg_w_gap_femur'

input_v_sacrum_path  = str((i_dir_vol /input_v_sacrum).with_suffix(i2_format))
input_v_lpelvis_path = str((i_dir_vol /input_v_lpelvis).with_suffix(i2_format))
input_v_rpelvis_path = str((i_dir_vol /input_v_rpelvis).with_suffix(i2_format))
input_v_lfemur_path  = str((i_dir_vol /input_v_lfemur).with_suffix(i2_format))
input_v_rfemur_path  = str((i_dir_vol /input_v_rfemur).with_suffix(i2_format))

# carts
input_v_lsi      = model_id + '_girdle_w_gap_lsi' 
input_v_rsi      = model_id + '_girdle_w_gap_rsi'
input_v_lpelvic  = model_id + '_girdle_w_gap_lpc'
input_v_rpelvic  = model_id + '_girdle_w_gap_rpc' 
input_v_lfemoral = model_id + '_left_leg_w_gap_cart' 
input_v_rfemoral = model_id + '_right_leg_w_gap_cart' 
input_v_pubic    = model_id + '_girdle_w_gap_pubic'   

input_v_lsi_path      = str((i_dir_vol /input_v_lsi).with_suffix(i2_format))
input_v_rsi_path      = str((i_dir_vol /input_v_rsi).with_suffix(i2_format))
input_v_lpelvic_path  = str((i_dir_vol /input_v_lpelvic).with_suffix(i2_format))
input_v_rpelvic_path  = str((i_dir_vol /input_v_rpelvic).with_suffix(i2_format))
input_v_lfemoral_path = str((i_dir_vol /input_v_lfemoral).with_suffix(i2_format))
input_v_rfemoral_path = str((i_dir_vol /input_v_rfemoral).with_suffix(i2_format))
input_v_pubic_path    = str((i_dir_vol /input_v_pubic).with_suffix(i2_format))

# implementation

In [5]:
s1_vertices, s1_faces = src.read (input_s_sacrum_path,  i_dim ) 
s2_vertices, s2_faces = src.read (input_s_lpelvis_path, i_dim )
s3_vertices, s3_faces = src.read (input_s_rpelvis_path, i_dim ) 
s4_vertices, s4_faces = src.read (input_s_lfemur_path,  i_dim )
s5_vertices, s5_faces = src.read (input_s_rfemur_path,  i_dim ) 
s6_vertices, s6_faces = src.read (input_s_lsi_path,  i_dim ) 
s7_vertices, s7_faces = src.read (input_s_rsi_path, i_dim )
s8_vertices, s8_faces = src.read (input_s_lpelvic_path, i_dim ) 
s9_vertices, s9_faces = src.read (input_s_rpelvic_path,  i_dim )
s10_vertices, s10_faces = src.read (input_s_lfemoral_path,  i_dim ) 
s11_vertices, s11_faces = src.read (input_s_rfemoral_path,  i_dim )
s12_vertices, s12_faces = src.read (input_s_pubic_path,  i_dim ) 

number of triangles after reading 44510
number of triangles after reading 36084
number of triangles after reading 35832
number of triangles after reading 45274
number of triangles after reading 47032
number of triangles after reading 4076
number of triangles after reading 3528
number of triangles after reading 2848
number of triangles after reading 3038
number of triangles after reading 17606
number of triangles after reading 17756
number of triangles after reading 468


In [6]:
merge_s_vertices = np.concatenate((s1_vertices,s2_vertices,s3_vertices,s4_vertices,
                                   s5_vertices,s6_vertices,s7_vertices,s8_vertices,
                                   s9_vertices,s10_vertices,s11_vertices,s12_vertices))

merge_s_faces = np.concatenate((s1_faces, s2_faces+len(s1_vertices),
                                   s3_faces +len(s1_vertices)+len(s2_vertices),
                                   s4_faces +len(s1_vertices)+len(s2_vertices)+len(s3_vertices),
                                   s5_faces + len(s1_vertices)+len(s2_vertices)+len(s3_vertices)+len(s4_vertices),
                                   s6_faces + len(s1_vertices)+len(s2_vertices)+len(s3_vertices)+len(s4_vertices)+len(s5_vertices),
                                   s7_faces + len(s1_vertices)+len(s2_vertices)+len(s3_vertices)+len(s4_vertices)+len(s5_vertices)+len(s6_vertices),
                                   s8_faces + len(s1_vertices)+len(s2_vertices)+len(s3_vertices)+len(s4_vertices)+len(s5_vertices)+len(s6_vertices)+len(s7_vertices),
                                   s9_faces + len(s1_vertices)+len(s2_vertices)+len(s3_vertices)+len(s4_vertices)+len(s5_vertices)+len(s6_vertices)+len(s7_vertices)+len(s8_vertices),
                                   s10_faces + len(s1_vertices)+len(s2_vertices)+len(s3_vertices)+len(s4_vertices)+len(s5_vertices)+len(s6_vertices)+len(s7_vertices)+len(s8_vertices)+len(s9_vertices),
                                   s11_faces + len(s1_vertices)+len(s2_vertices)+len(s3_vertices)+len(s4_vertices)+len(s5_vertices)+len(s6_vertices)+len(s7_vertices)+len(s8_vertices)+len(s9_vertices)+len(s10_vertices),
                                   s12_faces + len(s1_vertices)+len(s2_vertices)+len(s3_vertices)+len(s4_vertices)+len(s5_vertices)+len(s6_vertices)+len(s7_vertices)+len(s8_vertices)+len(s9_vertices)+len(s10_vertices)+len(s11_vertices)
                                  ))

In [7]:
v1_vertices, v1_faces = src.read (input_v_sacrum_path,  i_dim ) 
v2_vertices, v2_faces = src.read (input_v_lpelvis_path, i_dim )
v3_vertices, v3_faces = src.read (input_v_rpelvis_path, i_dim ) 
v4_vertices, v4_faces = src.read (input_v_lfemur_path,  i_dim )
v5_vertices, v5_faces = src.read (input_v_rfemur_path,  i_dim ) 
v6_vertices, v6_faces = src.read (input_v_lsi_path,  i_dim ) 
v7_vertices, v7_faces = src.read (input_v_rsi_path, i_dim )
v8_vertices, v8_faces = src.read (input_v_lpelvic_path, i_dim ) 
v9_vertices, v9_faces = src.read (input_v_rpelvic_path,  i_dim )
v10_vertices, v10_faces = src.read (input_v_lfemoral_path,  i_dim ) 
v11_vertices, v11_faces = src.read (input_v_rfemoral_path,  i_dim )
v12_vertices, v12_faces = src.read (input_v_pubic_path,  i_dim ) 

number of triangles after reading 20554
number of triangles after reading 23340
number of triangles after reading 22930
number of triangles after reading 44272
number of triangles after reading 46312
number of triangles after reading 3426
number of triangles after reading 2110
number of triangles after reading 1796
number of triangles after reading 1884
number of triangles after reading 18392
number of triangles after reading 18606
number of triangles after reading 438


In [8]:
merge_v_vertices = np.concatenate((v1_vertices,v2_vertices,v3_vertices,v4_vertices,
                                   v5_vertices,v6_vertices,v7_vertices,v8_vertices,
                                   v9_vertices,v10_vertices,v11_vertices,v12_vertices))

merge_v_faces = np.concatenate((v1_faces, v2_faces+len(v1_vertices),
                                   v3_faces +len(v1_vertices)+len(v2_vertices),
                                   v4_faces +len(v1_vertices)+len(v2_vertices)+len(v3_vertices),
                                   v5_faces + len(v1_vertices)+len(v2_vertices)+len(v3_vertices)+len(v4_vertices),
                                   v6_faces + len(v1_vertices)+len(v2_vertices)+len(v3_vertices)+len(v4_vertices)+len(v5_vertices),
                                   v7_faces + len(v1_vertices)+len(v2_vertices)+len(v3_vertices)+len(v4_vertices)+len(v5_vertices)+len(v6_vertices),
                                   v8_faces + len(v1_vertices)+len(v2_vertices)+len(v3_vertices)+len(v4_vertices)+len(v5_vertices)+len(v6_vertices)+len(v7_vertices),
                                   v9_faces + len(v1_vertices)+len(v2_vertices)+len(v3_vertices)+len(v4_vertices)+len(v5_vertices)+len(v6_vertices)+len(v7_vertices)+len(v8_vertices),
                                   v10_faces + len(v1_vertices)+len(v2_vertices)+len(v3_vertices)+len(v4_vertices)+len(v5_vertices)+len(v6_vertices)+len(v7_vertices)+len(v8_vertices)+len(v9_vertices),
                                   v11_faces + len(v1_vertices)+len(v2_vertices)+len(v3_vertices)+len(v4_vertices)+len(v5_vertices)+len(v6_vertices)+len(v7_vertices)+len(v8_vertices)+len(v9_vertices)+len(v10_vertices),
                                   v12_faces + len(v1_vertices)+len(v2_vertices)+len(v3_vertices)+len(v4_vertices)+len(v5_vertices)+len(v6_vertices)+len(v7_vertices)+len(v8_vertices)+len(v9_vertices)+len(v10_vertices)+len(v11_vertices)
                                  ))

In [9]:
frame = mp.plot(merge_s_vertices, merge_s_faces, c = src.bone, shading = src.sh_false)
# frame.add_mesh (s2_vertices, s2_faces, c = src.bone, shading = src.sh_false)
# frame.add_mesh (s3_vertices, s3_faces, c = src.bone, shading = src.sh_false)
# frame.add_mesh (s4_vertices, s4_faces, c = src.bone, shading = src.sh_false)
# frame.add_mesh (s5_vertices, s5_faces, c = src.bone, shading = src.sh_false)
# frame.add_mesh (s6_vertices, s6_faces, c = src.pastel_blue, shading = src.sh_false)
# frame.add_mesh (s7_vertices, s7_faces, c = src.pastel_blue, shading = src.sh_false)
# frame.add_mesh (s8_vertices, s8_faces, c = src.pastel_blue, shading = src.sh_false)
# frame.add_mesh (s9_vertices, s9_faces, c = src.pastel_blue, shading = src.sh_false)
# frame.add_mesh (s10_vertices, s10_faces, c = src.pastel_blue, shading = src.sh_false)
# frame.add_mesh (s11_vertices, s11_faces, c = src.pastel_blue, shading = src.sh_false)
# frame.add_mesh (s12_vertices, s12_faces, c = src.pastel_blue, shading = src.sh_false)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(10.181289…

In [10]:
frame = mp.plot(merge_v_vertices, merge_v_faces, c = src.bone, shading = src.sh_false)
# frame.add_mesh (v2_vertices, v2_faces, c = src.bone, shading = src.sh_false)
# frame.add_mesh (v3_vertices, v3_faces, c = src.bone, shading = src.sh_false)
# frame.add_mesh (v4_vertices, v4_faces, c = src.bone, shading = src.sh_false)
# frame.add_mesh (v5_vertices, v5_faces, c = src.bone, shading = src.sh_false)
# frame.add_mesh (v6_vertices, v6_faces, c = src.pastel_blue, shading = src.sh_false)
# frame.add_mesh (v7_vertices, v7_faces, c = src.pastel_blue, shading = src.sh_false)
# frame.add_mesh (v8_vertices, v8_faces, c = src.pastel_blue, shading = src.sh_false)
# frame.add_mesh (v9_vertices, v9_faces, c = src.pastel_blue, shading = src.sh_false)
# frame.add_mesh (v10_vertices, v10_faces, c = src.pastel_blue, shading = src.sh_false)
# frame.add_mesh (v11_vertices, v11_faces, c = src.pastel_blue, shading = src.sh_false)
# frame.add_mesh (v12_vertices, v12_faces, c = src.pastel_blue, shading = src.sh_false)

# frame.add_mesh (s1_vertices, s1_faces, c = src.sweet_pink, shading = src.sh_false)
# frame.add_mesh (s2_vertices, s2_faces, c = src.sweet_pink, shading = src.sh_false)
# frame.add_mesh (s3_vertices, s3_faces, c = src.sweet_pink, shading = src.sh_false)
# frame.add_mesh (s4_vertices, s4_faces, c = src.sweet_pink, shading = src.sh_false)
# frame.add_mesh (s5_vertices, s5_faces, c = src.sweet_pink, shading = src.sh_false)
# frame.add_mesh (s6_vertices, s6_faces, c = src.sweet_pink, shading = src.sh_false)
# frame.add_mesh (s7_vertices, s7_faces, c = src.sweet_pink, shading = src.sh_false)
# frame.add_mesh (s8_vertices, s8_faces, c = src.sweet_pink, shading = src.sh_false)
# frame.add_mesh (s9_vertices, s9_faces, c = src.sweet_pink, shading = src.sh_false)
# frame.add_mesh (s10_vertices, s10_faces, c = src.sweet_pink, shading = src.sh_false)
# frame.add_mesh (s11_vertices, s11_faces, c = src.sweet_pink, shading = src.sh_false)
# frame.add_mesh (s1_vertices, s1_faces, c = src.sweet_pink, shading = src.sh_false)



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(10.177055…

In [11]:
d= igl.hausdorff(merge_s_vertices,merge_s_faces,merge_v_vertices, merge_v_faces)
print('HD in total:  ', np.round(d,2), 'mm')
bb = igl.bounding_box_diagonal(merge_s_vertices)
per = d*100/bb
print('Percentage of the sacrum bounding box:', np.round(per,2),'%')
##

HD in total:   0.18 mm
Percentage of the sacrum bounding box: 0.04 %


In [12]:
# d1= igl.hausdorff(s1_vertices,s1_faces,v1_vertices, v1_faces)
# print('HD in sacrum:  ', np.round(d1,2), 'mm')
# bb1 = igl.bounding_box_diagonal(s1_vertices)
# per1 = d1*100/bb1
# print('Percentage of the sacrum bounding box:', np.round(per1,2),'%')
# ##

# d2= igl.hausdorff(s2_vertices,s2_faces,v2_vertices, v2_faces)
# print('HD in lpelvis: ',np.round(d2,2), 'mm')
# bb2 = igl.bounding_box_diagonal(s2_vertices)
# per2 = d2*100/bb2
# print('Percentage of the lpelvis bounding box:', np.round(per2,2),'%')
# ##

# d3= igl.hausdorff(s3_vertices,s3_faces,v3_vertices, v3_faces)
# print('HD in rpelvis: ',np.round(d3,2), 'mm')
# bb3 = igl.bounding_box_diagonal(s3_vertices)
# per3 = d3*100/bb3
# print('Percentage of the rpelvis bounding box:', np.round(per3,2),'%')
# ##

# d4= igl.hausdorff(s4_vertices,s4_faces,v4_vertices, v4_faces)
# print('HD in lfemur:  ',np.round(d4,2), 'mm')
# bb4 = igl.bounding_box_diagonal(s4_vertices)
# per4 = d4*100/bb4
# print('Percentage of the lfemur bounding box:', np.round(per4,2),'%')
# ##

# d5= igl.hausdorff(s5_vertices,s5_faces,v5_vertices, v5_faces)
# print('HD in rfemur:  ', np.round(d5,2), 'mm')
# bb5 = igl.bounding_box_diagonal(s5_vertices)
# per5 = d5*100/bb5
# print('Percentage of the rfemur bounding box:', np.round(per5,2),'%')
# ## 

# d6= igl.hausdorff(s6_vertices,s6_faces,v6_vertices, v6_faces)
# print('HD in lsi:    ', np.round(d6,2), 'mm')
# bb6 = igl.bounding_box_diagonal(s6_vertices)
# per6 = d6*100/bb6
# print('Percentage of the lsi bounding box:', np.round(per6,2),'%')

# d7= igl.hausdorff(s7_vertices,s7_faces,v7_vertices, v7_faces)
# print('HD in rsi:     ', np.round(d7,2), 'mm')
# bb7 = igl.bounding_box_diagonal(s7_vertices)
# per7 = d7*100/bb7
# print('Percentage of the rsi bounding box:', np.round(per7,2),'%')

# d8= igl.hausdorff(s8_vertices,s8_faces,v8_vertices, v8_faces)
# print('HD in lpc:     ', np.round(d8,2), 'mm')
# bb8 = igl.bounding_box_diagonal(s8_vertices)
# per8 = d8*100/bb8
# print('Percentage of the lpc bounding box:', np.round(per8,2),'%')

# d9= igl.hausdorff(s9_vertices,s9_faces,v9_vertices, v9_faces)
# print('HD in rpc:     ', np.round(d9,2), 'mm')
# bb9 = igl.bounding_box_diagonal(s9_vertices)
# per9 = d9*100/bb9
# print('Percentage of the rpc bounding box:', np.round(per9,2),'%')

# d10= igl.hausdorff(s10_vertices,s10_faces,v10_vertices, v10_faces)
# print('HD in lfc:     ', np.round(d10,2), 'mm')
# bb10 = igl.bounding_box_diagonal(s10_vertices)
# per10 = d10*100/bb10
# print('Percentage of the lfc bounding box:', np.round(per10,2),'%')

# d11= igl.hausdorff(s11_vertices,s11_faces,v11_vertices, v11_faces)
# print('HD in rfc:     ', np.round(d11,2), 'mm')
# bb11 = igl.bounding_box_diagonal(s11_vertices)
# per11 = d11*100/bb11
# print('Percentage of the rfc bounding box:', np.round(per11,2),'%')

# d12= igl.hausdorff(s12_vertices,s12_faces,v12_vertices, v12_faces)
# print('HD in pubic:     ', np.round(d12,2),'mm')
# bb12 = igl.bounding_box_diagonal(s12_vertices)
# per12 = d12*100/bb12
# print('Percentage of the pubic bounding box:', np.round(per12,2),'%')

In [13]:
# sd1,_,_= igl.signed_distance(s1_vertices,v1_vertices, v1_faces)
# print('HD in sacrum:  ', np.max(sd1), 'mm')

# sd2,_,_= igl.signed_distance(v1_vertices,s1_vertices, s1_faces)
# print('HD in sacrum:  ', np.max(sd2), 'mm')

# ##

In [14]:
# d1= igl.hausdorff(s1_vertices,s1_faces,v1_vertices, v1_faces)
# print(d1)

In [15]:
# a = np.array([172.50,165.14,167.70,171.46,180.79,169.86,169.86,192.73,185.21,174.39,187.36])
# b = np.array([153.67,143.00,146.99,143.74,151.50,142.01,132.31,161.85,157.23,153.46,152.92])
# c = np.array([156.55,146.14,143.78,140.53,147.85,138.38,130.94,155.12,151.38,151.75,151.21])
# d = np.array([58.83,52.07,36.94,49.67,60.41,40.84,48.55,54.29,58.65,50.77,46.33])
# e = np.array([61.01,46.15,47.81,37.35,68.32,53.77,44.90,58.78,54.73,54.87,49.57])
# f = np.array([290.29,263.16,253.50,258.64,309.08,264.45,263.20,305.96,298.58,278.11,283.22])
# g = np.array([26.94,24.99,28.06,25.49,27.99,25.31,27.25,26.50,28.30,25.82,26.67])
# h = np.array([27.85,24.87,26.20,26.19,27.54,26.49,23.63,27.78,28.54,25.62,27.08])
# i = np.array([26.00,24.00,25.00,25.00,27.00,24.00,25.00,25.00,27.00,25.00,25.00])
# j = np.array([27.00,24.00,24.00,25.00,27.00,25.00,23,27,27,25,26])
# k = np.array([233.54,191.11,166.63,167.73,201.21,129.86,92.03,153.37,110.55,90.75,97.84])
# l = np.array([228.31,195.21,160.76,184.26,190.27,122.29,92.50,157.99,113.15,100.11,92.82])
std = np.std(l)
mean = np.mean(l)

print('mean',np.round(mean,2))
print('standar daviation', np.round(std,2))


NameError: name 'l' is not defined

In [ ]:
# t = np.array([2.32,2.59, 2.53, 2.46, 3.01, 4.02, 4.43, 3.46, 4.33, 5.19, 3.49])
# std = np.std(t)
# mean = np.mean(t)
# print('mean',mean)
# print('standar daviation', std)



In [29]:
# a = np.array([2.0,2.2,2.6,2.6,2.5,3.6,5.2,3.2,4.6,5.3,4.8])
# a = np.array([192,264,260,223,204,274,341,251,326,326,299])
# a = np.array([0.14,0.30,0.13,0.13,0.19,0.15,0.14,0.18,0.15,0.15,0.14])
# a = np.array([0.14,0.17,0.26,0.14,0.28,0.14,0.14,0.16,0.17,0.15,0.16])
# a = np.array([0.03,0.07,0.03,0.03,0.04,0.04,0.04,0.04,0.03,0.03,0.03])
a = np.array([0.03,0.04,0.06,0.03,0.06,0.03,0.04,0.03,0.04,0.03,0.04])
# a = np.array([70.26,70.28,70.20,70.27,70.29,70.28,70.29,70.27,70.28,70.32,70.30])
# a = np.array([70.36,70.35,70.34,70.36,70.32,70.30,70.26,70.35,70.28,70.32,70.29])
# a= np.array([17.45,17.09,17.18,17.21,17.59,17.61,17.23,17.40,17.54,17.50,17.26])
# a= np.array([21.41,19.61,21.08,20.97,20.55,20.16,19.14,20.71,19.58,19.88,19.95])
# a= np.array([0.03,0.03,0.04,0.01,0.02,0.01,0.01,0.01,0.02,0.01,0.01])
# a=np.array([0.84,0.44,0.40,0.44,0.62,0.46,0.44,0.47,0.63,0.36,0.63])
# a = np.array([0,0,0,0,0,0,0,0,0,0,0])
# a = np.array([0.08,0,0.17,0.10,0.11,0.08,0.05,0.12,0.09,0.06,0.08])
std = np.std(a)
mean = np.mean(a)
print('mean',np.round(mean,2))
print('standar daviation', np.round(std,2))



mean 0.04
standar daviation 0.01


In [ ]:
# a = np.array([1.11,1.33,0.95,1.26,1.23,1.29,1.40,1.47,1.59,1.64,1.17])
# a = np.array([1.23,1.48,1.06,1.40,1.37,1.45,1.56,1.63,1.77,1.82,1.30 ])
# a = np.array([2914.16,2515.68,2236.36,2758.58,2998.12,2208.21,2198.43,2941.67,2819.23,2266.76,2240.89])
# a = np.array([1.33,1.58,1.05,1.43,1.44,1.36,1.56,1.54,1.62,1.41,1.28])
# a = np.array([1.61,1.83,1.17,1.61,1.64,1.54,1.86,1.75,2.01,1.56,1.43])
# a=np.array([5753.90,4808.44,4849.46,5465.50,5866.13,4831.51,4608.98,5325.95,6111.12,4950.50,5051.55])
# a = np.array([5.13,5.06,3.58,4.16,4.60,4.21,5.88,5.38,4.71,6.30,4.96])
# a = np.array([2401.30,1930.80,1696.37,1864.25,2701.15,1310.26,2048.43,2484.03,2332.81,2451.53,2245.62])
# a = np.array([2381.49,2028.47,1808.11,1982.89,2831.03,1419.71,2136.46,2552.02,2447.49,2550.57,2396.42])
# a = np.array([2.05,2.89,1.46,3.86,2.23,2.30,3.04,2.45,1.77,2.48,2.21])
# a = np.array([475.53, 338.92,268.58,411.19,517.00,337.80,352.19,351.36,461.99,559.05,365.42])
# a = np.array([492.63,301.79,268.96,416.17,538.50,309.89,369.18,353.73,494.80,555.34,340.60])
std = np.std(a)
mean = np.mean(a)
print('mean',np.round(mean,2))
print('standar daviation', np.round(std,2))



In [18]:
# a = np.array([0.024,0.024,0.024,0.024,0.024,0.024,0.024,0.024,0.024,0.024,0.022 ])
a = np.array([0.020,0.020,0.018,0.02,0.018,0.018,0.018,0.020,0.018,0.020,0.020])
std = np.std(a)
mean = np.mean(a)
print('mean',np.round(mean,3))
print('standar daviation', np.round(std,3))



mean 0.019
standar daviation 0.001
